In [323]:
import pandas as pd

df_clean = pd.read_parquet('../../data/processed/df_forecast.parquet')

In [324]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15410 entries, 0 to 15409
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   event_cultural   15410 non-null  float64
 1   event_national   15410 non-null  float64
 2   event_religious  15410 non-null  float64
 3   event_sport      15410 non-null  float64
 4   day_of_year      15410 non-null  int32  
 5   day_of_week      15410 non-null  int32  
 6   month            15410 non-null  int32  
 7   year             15410 non-null  int32  
 8   revenue          15410 non-null  float64
dtypes: float64(5), int32(4)
memory usage: 842.9 KB


In [325]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

rf = HistGradientBoostingRegressor(max_iter=10
                            , learning_rate=0.1
                            , max_depth=None
                            , max_leaf_nodes = 100
                            #, subsample = 0.9
                            , min_samples_leaf = 1
                            , n_iter_no_change = 500
                            , verbose=1)


#### Traing regression model in chunks

In [326]:
import pandas as pd
from sklearn.model_selection import train_test_split

target = df_clean.pop('revenue')
x_train, x_test, y_train, y_test = train_test_split(df_clean, target, test_size=0.2, random_state=8)

rf.fit(x_train, y_train)
print(rf.get_params())




Binning 0.001 GB of training data: 0.004 s
Binning 0.000 GB of validation data: 0.000 s
Fitting gradient boosted rounds:
[1/10] 1 tree, 100 leaves, max depth = 12, train loss: 6403058.81663, val loss: 6260463.37591, in 0.025s
[2/10] 1 tree, 100 leaves, max depth = 13, train loss: 6088558.33931, val loss: 5978342.61860, in 0.023s
[3/10] 1 tree, 100 leaves, max depth = 12, train loss: 5833615.88112, val loss: 5750527.12406, in 0.024s
[4/10] 1 tree, 100 leaves, max depth = 13, train loss: 5625411.74319, val loss: 5570969.12041, in 0.024s
[5/10] 1 tree, 100 leaves, max depth = 13, train loss: 5455711.36070, val loss: 5428756.71794, in 0.032s
[6/10] 

1 tree, 100 leaves, max depth = 13, train loss: 5316252.79131, val loss: 5311756.85521, in 0.026s
[7/10] 1 tree, 100 leaves, max depth = 13, train loss: 5202453.45007, val loss: 5216637.79396, in 0.033s
[8/10] 1 tree, 100 leaves, max depth = 14, train loss: 5107334.10975, val loss: 5140302.04591, in 0.024s
[9/10] 1 tree, 100 leaves, max depth = 15, train loss: 5029450.54981, val loss: 5083577.96845, in 0.029s
[10/10] 1 tree, 100 leaves, max depth = 16, train loss: 4961680.39556, val loss: 5041044.31291, in 0.028s
Fit 10 trees in 0.283 s, (1000 total leaves)
Time spent computing histograms: 0.100s
Time spent finding best splits:  0.035s
Time spent applying splits:      0.054s
Time spent predicting:           0.002s
{'categorical_features': None, 'early_stopping': 'auto', 'interaction_cst': None, 'l2_regularization': 0.0, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_bins': 255, 'max_depth': None, 'max_iter': 10, 'max_leaf_nodes': 100, 'min_samples_leaf': 1, 'monotonic_cst': None, 

In [327]:
from joblib import dump
dump(rf, '../../models/predictive/forecast.joblib')

['../../models/predictive/forecast.joblib']

In [328]:
x_test

,event_cultural,event_national,event_religious,event_sport,day_of_year,day_of_week,month,year
1550,0.0,0.0,0.0,0.0,4,4,1,2013
7599,0.0,0.0,1.0,0.0,55,6,2,2013
11780,0.0,0.0,0.0,0.0,253,6,9,2012
4640,0.0,0.0,0.0,0.0,6,1,1,2015
1483,0.0,1.0,0.0,0.0,147,0,5,2013
...,...,...,...,...,...,...,...,...
12794,0.0,0.0,0.0,0.0,111,4,4,2012
9939,0.0,0.0,0.0,0.0,174,6,6,2013
12582,0.0,0.0,0.0,0.0,64,5,3,2011
4108,0.0,0.0,0.0,0.0,262,0,9,2011


In [329]:
y_test

1550      7886.58
7599     10993.35
11780     5286.88
4640      7059.04
1483     13232.82
           ...   
12794     4252.80
9939     10258.36
12582     5566.16
4108     13376.16
1721      6430.20
Name: revenue, Length: 3082, dtype: float64

In [330]:
y_pred = rf.predict(x_test)

In [331]:
## RMSE evaluation
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

3204.5548005387946

In [332]:
# r2 score
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.23463326922257166